In [ ]:
# IMPORT ------------------------------------------------------------------------------------------

import os
import arcpy 
import numpy as np
import pandas as pd

from collections import Counter

pd.options.mode.chained_assignment = None  # default='warn'

# DEFINE PATHS ------------------------------------------------------------------------------------

base_path = os.path.abspath("..")
in_folder = os.path.join(base_path, "input")
in_MHN_GDB = os.path.join(in_folder, "MHN.gdb")

arcpy.env.workspace = in_MHN_GDB

## Highway

### Check if highway network is a multigraph

In [ ]:
fc = "hwynet/hwynet_arc"
fields = [f.name for f in arcpy.ListFields(fc) if f.type!="Geometry"] 
link_df = pd.DataFrame(data=arcpy.da.SearchCursor(fc, fields), columns=fields)

link_df["AB"] = link_df["ANODE"].astype("string") + "-" + link_df["BNODE"].astype("string")
link_df = link_df[["AB", "ABB", "ANODE", "BNODE", "DIRECTIONS", "ROADNAME"]]
# link_df.AB.value_counts()
link_df_rev = link_df.copy()
link_df_rev["ANODE"] = link_df["BNODE"]
link_df_rev["BNODE"] = link_df["ANODE"]
link_df_rev["AB"] = link_df_rev["ANODE"].astype("string") + "-" + link_df_rev["BNODE"].astype("string") 

# ah nuts. It is a multigraph. 

pd.merge(link_df, link_df_rev, how = "inner", on = "AB")

,AB,ABB_x,ANODE_x,BNODE_x,DIRECTIONS_x,ROADNAME_x,ABB_y,ANODE_y,BNODE_y,DIRECTIONS_y,ROADNAME_y
0,23507-23508,23507-23508-1,23507,23508,1,55TH ST,23508-23507-1,23507,23508,1,55TH ST
1,21322-16377,21322-16377-1,21322,16377,1,US 41 / LAKE SHORE DR SB,16377-21322-1,21322,16377,1,US 41 / LAKE SHORE DR NB
2,16042-21282,16042-21282-1,16042,21282,1,US 41 / LAKE SHORE DR NB,21282-16042-1,16042,21282,1,US 41 / LAKE SHORE DR SB
3,21279-21281,21279-21281-1,21279,21281,1,US 41 / LAKE SHORE DR SB,21281-21279-1,21279,21281,1,US 41 / LAKE SHORE DR NB
4,21282-16042,21282-16042-1,21282,16042,1,US 41 / LAKE SHORE DR SB,16042-21282-1,21282,16042,1,US 41 / LAKE SHORE DR NB
5,21281-21282,21281-21282-1,21281,21282,1,US 41 / LAKE SHORE DR SB,21282-21281-1,21281,21282,1,US 41 / LAKE SHORE DR NB
6,21282-21281,21282-21281-1,21282,21281,1,US 41 / LAKE SHORE DR NB,21281-21282-1,21282,21281,1,US 41 / LAKE SHORE DR SB
7,21281-21279,21281-21279-1,21281,21279,1,US 41 / LAKE SHORE DR NB,21279-21281-1,21281,21279,1,US 41 / LAKE SHORE DR SB
8,16377-21322,16377-21322-1,16377,21322,1,US 41 / LAKE SHORE DR NB,21322-16377-1,16377,21322,1,US 41 / LAKE SHORE DR SB
9,12775-12745,12775-12745-1,12775,12745,1,,12745-12775-1,12775,12745,1,


### Check for skeleton links which are later modified

In [75]:
fc = "hwyproj_coding"
fields = [f.name for f in arcpy.ListFields(fc) if f.type!="Geometry"] 
hwyproj_df = pd.DataFrame(data=arcpy.da.SearchCursor(fc, fields), columns=fields)

fc = "hwyproj"
fields = [f.name for f in arcpy.ListFields(fc) if f.type!="Geometry"] 
year_df = pd.DataFrame(data=arcpy.da.SearchCursor(fc, fields), columns=fields)

hwyproj_df = pd.merge(hwyproj_df, year_df, how = "left", on = "TIPID")

In [85]:
skeleton = hwyproj_df[(hwyproj_df.ABB.str.endswith("0")) & (hwyproj_df.COMPLETION_YEAR != 9999) & (hwyproj_df.TOD == "0")]
skeleton_abbs = skeleton.ABB.value_counts().sort_values()

len(skeleton_abbs[skeleton_abbs >= 2])

27